Download the dataset

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("grouplens/movielens-20m-dataset")

print("Path to dataset files:", path)

In [ ]:
import numpy
import pandas as pd

## <strong> 1.1 Data Preparation
Download the MovieLens dataset from here. After downloading, explore the dataset to understand the structure and identify any preprocessing steps needed.

<strong> There are in total 6 datasets, let's have a quick look at them

In [10]:
genome_scores = pd.read_csv("dataset/genome_scores.csv", index_col =0)
genome_scores.head(3)

,tagId,relevance
movieId,,
1,1,0.02500
1,2,0.02500
1,3,0.05775


In [11]:
genome_tags = pd.read_csv("dataset/genome_tags.csv", index_col = 0)
genome_tags.head(3)

,tag
tagId,
1,007
2,007 (series)
3,18th century


In [12]:
link = pd.read_csv("dataset/link.csv", index_col = 0)
link.head(3)

,imdbId,tmdbId
movieId,,
1,114709,862.0
2,113497,8844.0
3,113228,15602.0


In [ ]:
movie = pd.read_csv("dataset/movie.csv", index_col = 0)
movie.head(3)

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance


In [14]:
rating = pd.read_csv("dataset/rating.csv", index_col = 0)
rating.head(3)

,movieId,rating,timestamp
userId,,,
1,2,3.5,2005-04-02 23:53:47
1,29,3.5,2005-04-02 23:31:16
1,32,3.5,2005-04-02 23:33:39


In [15]:
tag = pd.read_csv("dataset/tag.csv", index_col = 0)
tag.head(3)

,movieId,tag,timestamp
userId,,,
18,4141,Mark Waters,2009-04-24 18:19:40
65,208,dark hero,2013-05-10 01:41:18
65,353,dark hero,2013-05-10 01:41:19
